In [2]:
import xgboost as xgb
from xgboost import XGBClassifier, plot_tree
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics, preprocessing
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,roc_auc_score,roc_curve, precision_recall_curve, average_precision_score, auc
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
import datetime as dt
from sklearn.linear_model import LogisticRegression

In [3]:
path = "example_dataset.csv"
df_all = pd.read_csv(path, index_col=0)
adhere_score = df_all
gwtg_hf_score = df_all

## ADHERE score

In [ ]:
countlist=['_0','_1', '_2', '_3']
count=0

for col in countlist:
    adhere_score['adhere_score'+ col]=(0.0212*adhere_score['BUN'+col]
                                       -0.0192*adhere_score['Systolic_bp'+col]
                                      +0.0131*adhere_score['Heart_Rate'+col]
                                      +0.0288*adhere_score['PatientAge']
                                      -4.72)
    print(adhere_score.groupby('ExpiredStatus')['adhere_score'+col].value_counts())


In [ ]:
#prob = odds / (1 + odds)

odds=np.exp(adhere_score['adhere_score_0'])
prob=odds/(1+odds)

adhere_score['adhere_score_0']=prob

final_score=adhere_score[['HospitalAdmissionTime','ef_low','ExpiredStatus','ef_category','adhere_score_0']]
final_score['adhere_score_0']=final_score['adhere_score_0'].fillna(0)
final_score.to_csv("adhere_score.csv", index=False)

## GWTG_HF score

In [ ]:
gwtg_hf_score['COPD_Score']=np.where(gwtg_hf_score['COPD']==1,2,
                                    np.where(gwtg_hf_score['COPD']==0,0,np.NaN))
gwtg_hf_score['COPD_Score'].value_counts()

print('White = 0, LatinX=1, Black = 2, Asian = 3, Unknown = 4, Indigenous = 5')

gwtg_hf_score['Black_Race_Score']=np.where(gwtg_hf_score['Race']==2,0,3)
gwtg_hf_score['Black_Race_Score'].value_counts()

In [ ]:
gwtg_hf_score['Age_Score']=np.where(gwtg_hf_score['PatientAge']<=19,0,
                             np.where(gwtg_hf_score['PatientAge']<=29,3,
                                     np.where(gwtg_hf_score['PatientAge']<=39,6,
                                             np.where(gwtg_hf_score['PatientAge']<=49,8,
                                                     np.where(gwtg_hf_score['PatientAge']<=59,11,
                                                             np.where(gwtg_hf_score['PatientAge']<=69,14,
                                                                     np.where(gwtg_hf_score['PatientAge']<=79,17,
                                                                             np.where(gwtg_hf_score['PatientAge']<=89,19,
                                                                                     np.where(gwtg_hf_score['PatientAge']<=99,22,
                                                                                             np.where(gwtg_hf_score['PatientAge']<=109,25,
                                                                                                     np.where(gwtg_hf_score['PatientAge']>=110,28,np.NaN)))))))))))

gwtg_hf_score['Age_Score'].value_counts()

In [ ]:
gwtg_hf_score['Heart_Rate_Score']=np.where(gwtg_hf_score['Heart_Rate_0']<=79,0,
                                          np.where(gwtg_hf_score['Heart_Rate_0']<=84,1,
                                                  np.where(gwtg_hf_score['Heart_Rate_0']<=89,3,
                                                          np.where(gwtg_hf_score['Heart_Rate_0']<=94,4,
                                                                  np.where(gwtg_hf_score['Heart_Rate_0']<=99,5,
                                                                          np.where(gwtg_hf_score['Heart_Rate_0']<=104,6,
                                                                                  np.where(gwtg_hf_score['Heart_Rate_0']>=105,8,np.NaN)))))))

gwtg_hf_score['Heart_Rate_Score'].value_counts()

In [ ]:
gwtg_hf_score['Sodium_Score']=np.where(gwtg_hf_score['Sodium_0']<=130,4,
                                      np.where(gwtg_hf_score['Sodium_0']<=133,3,
                                              np.where(gwtg_hf_score['Sodium_0']<=136,2,
                                                      np.where(gwtg_hf_score['Sodium_0']<=138,1,
                                                              np.where(gwtg_hf_score['Sodium_0']>=139,0,0)))))
gwtg_hf_score['Sodium_Score'].value_counts()

In [ ]:
gwtg_hf_score['BUN_Score']=np.where(gwtg_hf_score['BUN_0']<=9,0,
                                   np.where(gwtg_hf_score['BUN_0']<=19,2,
                                           np.where(gwtg_hf_score['BUN_0']<=29,4,
                                                   np.where(gwtg_hf_score['BUN_0']<=39,6,
                                                           np.where(gwtg_hf_score['BUN_0']<=49,8,
                                                                   np.where(gwtg_hf_score['BUN_0']<=59,9,
                                                                           np.where(gwtg_hf_score['BUN_0']<=69,11,
                                                                                   np.where(gwtg_hf_score['BUN_0']<=79,13,
                                                                                           np.where(gwtg_hf_score['BUN_0']<=89,15,
                                                                                                   np.where(gwtg_hf_score['BUN_0']<=99,17,
                                                                                                           np.where(gwtg_hf_score['BUN_0']<=109,19,
                                                                                                                   np.where(gwtg_hf_score['BUN_0']<=119,21,
                                                                                                                           np.where(gwtg_hf_score['BUN_0']<=129,23,
                                                                                                                                   np.where(gwtg_hf_score['BUN_0']<=139,25,
                                                                                                                                           np.where(gwtg_hf_score['BUN_0']<=149,27,
                                                                                                                                                   np.where(gwtg_hf_score['BUN_0']>=150,28,0))))))))))))))))
gwtg_hf_score['BUN_Score'].value_counts()

In [ ]:
gwtg_hf_score['Systolic_BP_Score']=np.where((gwtg_hf_score['Systolic_bp_0']>=50)&(gwtg_hf_score['Systolic_bp_0']<=59),28,
                                           np.where(gwtg_hf_score['Systolic_bp_0']<=69,26,
                                                   np.where(gwtg_hf_score['Systolic_bp_0']<=79,24,
                                                           np.where(gwtg_hf_score['Systolic_bp_0']<=89,23,
                                                                   np.where(gwtg_hf_score['Systolic_bp_0']<=99,21,
                                                                           np.where(gwtg_hf_score['Systolic_bp_0']<=109,19,
                                                                                   np.where(gwtg_hf_score['Systolic_bp_0']<=119,17,
                                                                                           np.where(gwtg_hf_score['Systolic_bp_0']<=129,15,
                                                                                                   np.where(gwtg_hf_score['Systolic_bp_0']<=139,13,
                                                                                                           np.where(gwtg_hf_score['Systolic_bp_0']<=149,11,
                                                                                                                   np.where(gwtg_hf_score['Systolic_bp_0']<=159,9,
                                                                                                                           np.where(gwtg_hf_score['Systolic_bp_0']<=169,8,
                                                                                                                                   np.where(gwtg_hf_score['Systolic_bp_0']<=179,6,
                                                                                                                                           np.where(gwtg_hf_score['Systolic_bp_0']<=189,4,
                                                                                                                                                    np.where(gwtg_hf_score['Systolic_bp_0']<=199,2,
                                                                                                                                                    	np.where(gwtg_hf_score['Systolic_bp_0']>=200,0,0))))))))))))))))

gwtg_hf_score['Systolic_BP_Score'].value_counts()

In [ ]:
gwtg_hf_score['GWTG_HF_Score_Final']=(gwtg_hf_score['Age_Score']+gwtg_hf_score['Black_Race_Score']
                                      +gwtg_hf_score['COPD_Score']+gwtg_hf_score['Heart_Rate_Score']
                                     +gwtg_hf_score['Sodium_Score']+gwtg_hf_score['BUN_Score']+
                                     gwtg_hf_score['Systolic_BP_Score'])
gwtg_hf_score['GWTG_HF_Score_Final'].value_counts()

In [ ]:
gwtg_hf_score['GWTG_HF_Score_Category']=np.where(gwtg_hf_score['GWTG_HF_Score_Final']<=33,'<1%',
                                                np.where(gwtg_hf_score['GWTG_HF_Score_Final']<=50,'1-5%',
                                                        np.where(gwtg_hf_score['GWTG_HF_Score_Final']<=57,'>5-10%',
                                                                np.where(gwtg_hf_score['GWTG_HF_Score_Final']<=61,'>10-15%',
                                                                        np.where(gwtg_hf_score['GWTG_HF_Score_Final']<=65,'>15-20%',
                                                                                np.where(gwtg_hf_score['GWTG_HF_Score_Final']<=70,'>20-30%',
                                                                                        np.where(gwtg_hf_score['GWTG_HF_Score_Final']<=74,'>30-40%',
                                                                                                np.where(gwtg_hf_score['GWTG_HF_Score_Final']<=78,'>40-50%',
                                                                                                        np.where(gwtg_hf_score['GWTG_HF_Score_Final']>=79,'>50%','Not Available')))))))))
gwtg_hf_score['GWTG_HF_Score_Category'].value_counts()

In [ ]:
final_score=gwtg_hf_score[['HospitalAdmissionTime','ef_low','ExpiredStatus','ef_category','GWTG_HF_Score_Final',]]
final_score.to_csv("gwtg_hf_score.csv", index=False)